<a href="https://colab.research.google.com/github/saravanan-nj/notebooks/blob/main/qlora-tiny-llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Requirements

In [3]:
!pip install -q accelerate==1.3.0 peft==0.14.0 bitsandbytes==0.45.2 transformers==4.48.3 trl==0.14.0 datasets==3.1.0 pretty_midi # fsspec==2024.10.0

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoModel,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM

In [5]:
# Load the training dataset
from google.colab import drive
import pandas as pd

df = pd.read_csv('/content/imdb/train_data.txt', delimiter=":::", names=["index", "title", "genre", "description"])

<ipython-input-5-dbc0b2345c43>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/imdb/train_data.txt', delimiter=":::", names=["index", "title", "genre", "description"])


In [6]:
dataset_df = pd.DataFrame()
def get_text(row):
  return f"""
### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
{row["description"]}

### Genre:
{row["genre"].strip()}
<|endresponse>
"""
dataset_df["description"] = df["description"]
dataset_df["genre"] = df["genre"]
dataset_df["instruction"] = "Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie."
# dataset_df["text"] = dataset_df.apply(get_text, axis=1)

In [30]:
from datasets import Dataset
dataset = Dataset.from_pandas(dataset_df)

In [31]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [32]:
model_name = "crumb/nano-mistral"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [33]:
device_map = "auto"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [34]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = SFTConfig(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=100,
    learning_rate=1e-5,
    weight_decay=0.01,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    max_seq_length=256,
    packing=False
)

In [ ]:
def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)))


def formatted_prompts_func(example):
  output_texts = []
  for i in range(len(example["description"])):
    output_texts.append(f"""
### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
{example["description"][i]}

### Genre:
{example["genre"][i].strip()}
<|endresponse>
""")
  # print_tokens_with_ids(output_texts[0])
  return output_texts

response_template =[27332, 6242, 267, 28747, 13]  # "### Genre:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_arguments,
    processing_class=tokenizer,
    formatting_func=formatted_prompts_func,
    # data_collator=collator
)
trainer.train()
trainer.model.save_pretrained("imdb-classifier")

Map:   0%|          | 0/54214 [00:00<?, ? examples/s]

Step,Training Loss
100,3.663000
200,3.676500
300,3.601800
400,3.512900
500,3.374900
600,3.248400
700,3.141600
800,3.027600
900,2.909300
1000,2.795400


Step,Training Loss
100,3.663000
200,3.676500
300,3.601800
400,3.512900
500,3.374900
600,3.248400
700,3.141600
800,3.027600
900,2.909300
1000,2.795400


In [ ]:
!zip -r imdb-classifier.zip results imdb-classifier

updating: results/ (stored 0%)
updating: results/checkpoint-2390/ (stored 0%)
updating: results/checkpoint-2390/adapter_model.safetensors (deflated 7%)
updating: results/checkpoint-2390/README.md (deflated 66%)
updating: results/checkpoint-2390/training_args.bin (deflated 51%)
updating: results/checkpoint-2390/trainer_state.json (deflated 73%)
updating: results/checkpoint-2390/special_tokens_map.json (deflated 73%)
updating: results/checkpoint-2390/optimizer.pt (deflated 8%)
updating: results/checkpoint-2390/adapter_config.json (deflated 54%)
updating: results/checkpoint-2390/rng_state.pth (deflated 25%)
updating: results/checkpoint-2390/scheduler.pt (deflated 56%)
updating: results/checkpoint-2390/tokenizer.json (deflated 85%)
updating: results/checkpoint-2390/tokenizer.model (deflated 55%)
updating: results/checkpoint-2390/tokenizer_config.json (deflated 69%)
updating: results/runs/ (stored 0%)
updating: results/runs/Feb09_13-18-11_41e14a874267/ (stored 0%)
updating: results/runs/Feb

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(base_model, '/content/imdb-classifier')
model = model.merge_and_unload()
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, truncation=True)
original_pipeline = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=200, truncation=True)

prompt_template = """
### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
{}

### Genre:
"""

Device set to use cuda:0
Device set to use cuda:0


In [ ]:
movie_description = "Müzeyyen is a young woman who lives a drug-fuelled, chaotic life outside the norms of society. Her brother Ali interrupts his own successful, bourgeois life in Bolu, and goes to Antalya to meet with his recently- discovered sister. There, the legitimate child - Ali - will be tested through unguessed - at sins, while the illegitimate one - Müzeyyen - will be tempted in turn by Ali's hope."
print(len(movie_description))
movie_input = prompt_template.format(movie_description)
result = pipe(movie_input)
original_result = original_pipeline(movie_input)
print(result[0]["generated_text"])
print("*" * 60)
print(original_result[0]["generated_text"])

390

### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
Müzeyyen is a young woman who lives a drug-fuelled, chaotic life outside the norms of society. Her brother Ali interrupts his own successful, bourgeois life in Bolu, and goes to Antalya to meet with his recently- discovered sister. There, the legitimate child - Ali - will be tested through unguessed - at sins, while the illegitimate one - Müzeyyen - will be tempted in turn by Ali's hope.

### Genre:

German

Genre:

German

Genre:

German

Genre:

German

Genre:

German

Gen
************************************************************

### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
Müzeyyen is a young woman who lives a drug-fuelled, chaotic l

In [ ]:
movie_description = "Shy bookworm Tommy vacations in Palm Springs, never expecting an amorous adventure. But once he spots Brendan, he can't stop fantasizing about being with the sensuous, dark-haired man. However, Brendan's friends have their own ideas, and continue to pull him into their orgy of alcohol and sex. Finally alone at the pool, Brendan approaches Tommy and they spend an afternoon exploring the desert town and each other. Just as Tommy saves Brendan from a poolside accident, Brendan's friends come back - and try to force the two apart. True love conquers all, as Tommy's vacation climaxes in a passionate declaration of love and desire. LOVE INN EXILE boasts several unforgettable erotic scenes - including the strangely tender, drunken ménage a trois between Brendan's friends, a spicy tequila shot off Brendan's shoulder, and, of course, Tommy's ultimate fulfillment by his ideal man. LOVE INN EXILE is filmed entirely on location at Inn Exile in Palm Springs. With music by adult film star Sharon Kane, this video makes exile seem like a perfectly good punishment!"
movie_input = prompt_template.format(movie_description[:450])
result = pipe(movie_input)
original_result = original_pipeline(movie_input)
print(result[0]["generated_text"])
print("*" * 60)
print(original_result[0]["generated_text"])


### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
Shy bookworm Tommy vacations in Palm Springs, never expecting an amorous adventure. But once he spots Brendan, he can't stop fantasizing about being with the sensuous, dark-haired man. However, Brendan's friends have their own ideas, and continue to pull him into their orgy of alcohol and sex. Finally alone at the pool, Brendan approaches Tommy and they spend an afternoon exploring the desert town and each other. Just as Tommy saves Brendan from 

### Genre:

Gregory Goulding

Gregory Goulding is a writer and writer who has written for the New York Times, the New York Times, the New York Times, the New York Times
************************************************************

### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on